In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import GridSearchCV,KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder , OneHotEncoder 
from sklearn.metrics import mean_squared_log_error

# np.sqrt(mean_squared_log_error(actual, predicted))

import xgboost as xgb

In [ ]:
train_df =  pd.read_csv('/kaggle/input/machinehack-buyers-time-prediction-challenge/ParticipantData_BTPC/Train.csv')
test_df = pd.read_csv('/kaggle/input/machinehack-buyers-time-prediction-challenge/ParticipantData_BTPC/Test.csv')

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
len(pd.unique(train_df['session_id']))==len(train_df)

In [ ]:
train = train_df.copy()
test = test_df.copy()

# **BASIC EDA**

In [ ]:
train = train.drop(labels=['session_id','client_agent'],axis = 1)
test = test.drop(labels=['session_id','client_agent'],axis = 1)

In [ ]:
train['device_details'].value_counts()

In [ ]:
train.describe()

In [ ]:
train['session_number'].value_counts()

In [ ]:
train["date"] = pd.to_datetime(train['date'])
test["date"] = pd.to_datetime(test['date'])

train["year"] = train['date'].dt.year
test["year"] = test['date'].dt.year

train["month"] = train['date'].dt.month
test["month"] = test['date'].dt.month

train['day']= train['date'].dt.day
test["day"] = test['date'].dt.day

train['diff_date'] = ((train['date'] - pd.datetime(1970,1,1)).dt.total_seconds())/(10**6)
test['diff_date'] = ((test['date'] - pd.datetime(1970,1,1)).dt.total_seconds())/(10**6)

In [ ]:
train.head()

In [ ]:
l = pd.unique(train["device_details"])
device_encoding = {}
x = 0
for i in l:
    if i not in device_encoding:
        device_encoding[i] = x
        x+=1
print(device_encoding)

train["device_encoding"] = train["device_details"].map(device_encoding)
test["device_encoding"] = test["device_details"].map(device_encoding)

In [ ]:
train.head()

In [ ]:
train= train[pd.notnull(train["time_spent"])]

# **MODELLING**

In [ ]:
# cols = ['session_number','device_encoding','purchased','added_in_cart','checked_out','year','month','day','diff_date']
cols = ['session_number','purchased','added_in_cart','checked_out']
X = train[cols]
Y = train["time_spent"]

In [ ]:
Y

In [ ]:
x_tr,x_tst,y_tr,y_tst = train_test_split(X,Y,test_size=0.3,random_state=6969)

In [ ]:
xgr = xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6594975539639095, gamma=0,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             learning_rate=0.01634714303249965, max_delta_step=0, max_depth=9,
             min_child_weight=1, monotone_constraints='()',
             n_estimators=200, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             subsample=0.42320618378477715, tree_method='exact',
             validate_parameters=1, verbosity=2)
xgr.fit(X=x_tr[cols],y=y_tr,eval_set=[(x_tr[cols],y_tr)],eval_metric="rmse",verbose=2)
ans = abs(xgr.predict(x_tst))
score = np.sqrt(mean_squared_log_error(y_tst, ans))
score

In [ ]:
train.corr().time_spent

In [ ]:
feat_importances = pd.Series(xgr.feature_importances_, index=cols)
feat_importances.nlargest(6).plot(kind='barh')
plt.show()

In [ ]:
xgr.fit(X=X[cols],y=Y,verbose=2)
ans = xgr.predict(test[cols])

# **FILE CREATION**

In [ ]:
sub = pd.read_csv('../input/machinehack-buyers-time-prediction-challenge/ParticipantData_BTPC/Sample Submission.csv')
sub["time_spent"]=abs(ans)

In [ ]:
sub.to_csv('sub1_xgb1.csv', index=False)